In [1]:
import pickle
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint

# Exercise 4: Optimization

Now we have developed ML models for A and Rm - so let's use them to find the optimal composition to reach our target: Rm $\ge$ 1000 MPa and A $\ge$ 25 %. 

First, load the scaler and ML models used in the previous exercise. 

Then, define a function `fun` to be minimized by `scipy.optimize.minimize`. 

Afterwards, determine the initial guess `x0` for the minimization problem.

Now, run the optimization of the function `fun` using the optimization method `trust-constr`. Keep in mind, that the sum of elements should be 100 wt.% and that the concentration of each element can vary between 0 and 100 (wt.%).

In [3]:
scaler = pickle.load(open("scaler.pickle", "rb"))
model_A = pickle.load(open("model_A.pickle", "rb"))
model_Rm = pickle.load(open("model_Rm.pickle", "rb"))


In [4]:
def fun(composition):
    _composition = scaler.transform([composition])
    return - model_A.predict(_composition) * model_Rm.predict(_composition)**4

In [7]:
res = np.inf
for i in range(model_A.best_estimator_.n_support_[0]):
    _x0 = model_A.best_estimator_.support_vectors_[i]
    _res = fun(scaler.inverse_transform([_x0])[0])
    if _res < res:
        res = _res
        x0 = _x0
print(scaler.inverse_transform([x0]))
print(res)
print(model_A.predict([x0]))
print(model_Rm.predict([x0]))

[[7.968e+01 2.000e-02 0.000e+00 1.550e+01 4.800e+00 0.000e+00 0.000e+00
  0.000e+00 3.400e+00 0.000e+00]]
[-3.31978413e+13]
[22.68283581]
[1099.89980219]


In [10]:
popt = minimize(fun, x0=scaler.inverse_transform([x0])[0],
                method="trust-constr", bounds=[(0,100)]*len(x0),
                constraints=LinearConstraint(np.array([1.]*10), 99, 101))

In [11]:
popt

 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 1948
      cg_stop_cond: 2
            constr: [array([101.]), array([7.76656809e+01, 1.97536631e-02, 1.29536891e-03, 1.54798052e+01,
       4.42843196e+00, 3.49916829e-11, 3.38431428e-11, 3.87025937e-02,
       3.35828762e+00, 8.04267040e-03])]
       constr_nfev: [0, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 8.853141069412231
               fun: array([-3.27423485e+13])
              grad: array([-7.46317351e+11, -7.46151739e+11, -7.46312172e+11, -7.46317084e+11,
       -7.46315570e+11,  1.45320758e+12,  1.50112803e+12, -7.46315579e+11,
       -7.46315000e+11, -7.46310599e+11])
               jac: [array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]), array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       

In [15]:
opt_x = {el: round(xi, 2) for el, xi in zip(['c(Fe) (%)', 'c(C) (%)', 'c(N) (%)', 'c(Cr) (%)', 'c(Ni) (%)',
       'c(Mo) (%)', 'c(Al) (%)', 'c(Mn) (%)', 'c(Cu) (%)', 'c(Si) (%)'], popt.x)}
print(opt_x)
x_ = scaler.transform([popt.x])
print("A", model_A.predict(x_))
print("Rm", model_Rm.predict(x_))

{'c(Fe) (%)': 77.67, 'c(C) (%)': 0.02, 'c(N) (%)': 0.0, 'c(Cr) (%)': 15.48, 'c(Ni) (%)': 4.43, 'c(Mo) (%)': 0.0, 'c(Al) (%)': 0.0, 'c(Mn) (%)': 0.04, 'c(Cu) (%)': 3.36, 'c(Si) (%)': 0.01}
A [24.18611903]
Rm [1078.66331869]


{'c(Fe) (%)': 77.67,
 'c(C) (%)': 0.02,
 'c(N) (%)': 0.0,
 'c(Cr) (%)': 15.48,
 'c(Ni) (%)': 4.43,
 'c(Mo) (%)': 0.0,
 'c(Al) (%)': 0.0,
 'c(Mn) (%)': 0.04,
 'c(Cu) (%)': 3.36,
 'c(Si) (%)': 0.01}